In [1]:
import polars as pl
import torch
from torcheval.metrics.functional import multiclass_accuracy
# We are using the titanic dataset to learn about torch and neural networks, not because it is particulary intresting or neural networks are suited for this task.

In [2]:
#load the data and transform into torch tensors.
batch_size = 4
feature_columns = ["Sex", "Age", "Pclass"]

train_df = pl.read_parquet("train.parquet")
train_df = train_df.select(feature_columns + ["Survived"])
train_dataset = train_df.to_torch(return_type="dataset", label="Survived", features=feature_columns)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

eval_df = pl.read_parquet("eval.parquet")
eval_df = eval_df.select(feature_columns + ["Survived"])
eval_dataset = eval_df.to_torch(return_type="dataset", label="Survived", features=feature_columns)
eval_loader = torch.utils.data.DataLoader(dataset=eval_dataset, batch_size=batch_size)

In [3]:
class LinearRegressor(torch.nn.Module):
    def __init__(self, in_features: int):
        super().__init__()
        self.regressor = torch.nn.Linear(in_features=in_features, out_features=2)
        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.regressor(x)
        x = self.softmax(x)
        return x

class BasicNeuralNetwork(torch.nn.Module):
    def __init__(self, in_features: int):
        super().__init__()
        self.neural_network = torch.nn.Sequential(
            torch.nn.Linear(in_features, 4),
            torch.nn.ReLU(),
            torch.nn.Linear(4, 2),
            torch.nn.Softmax(dim=1),
        )
    
    def forward(self, x):
        x =  self.neural_network(x)
        return x

In [4]:
learning_rate = 0.01

model = LinearRegressor(in_features=3)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
epochs = 50

#Basic training loop
for epoch in range(epochs):    
    model.train() # activates training mode
    for features, labels in train_loader:
        optimizer.zero_grad() # zero gradients from previous step
        preds = model(features) # Make prediciton based on features
        loss = loss_fn(preds, labels) # calculates loss based on model prediciton and known truth (label)
        loss.backward() # calculate gradients based on loss
        optimizer.step() # Adjusts model parameters based on gradients
    print(epoch)

In [ ]:
# calculates the loss and the accuracy of the model for a given dataloader
def evaluate(data_loader: torch.utils.data.DataLoader):
    with torch.no_grad():
        model.eval()
        preds = []
        labels = []
        for features, label in data_loader:
            labels.append(label)
            preds.append(model(features))
        preds = torch.concat(preds)
        labels = torch.concat(labels)
        loss = loss_fn(preds, labels).item()
        accuracy = multiclass_accuracy(preds, labels).item()
        return {"loss": loss, "acc": accuracy}
    
# Example usage:
results_train = evaluate(train_loader)
print(results_train)
results_eval = evaluate(eval_loader)
print(results_eval)

In [7]:
# Task 1: 
# Evaluate on the train- and eval-Dataloader after every epoch.
# Tip: you have to move the cell with "evaluate" before the training loop.
# Tip: just copy and paste the example code after the second for loop.

In [ ]:
# Optinal Task 1.1 (python)
# Report the metrics from Task 1 in a prettier way using f-Strings. Also include the epoch.
# Example f-String:
x = 2
y = 3
print(f"{x} * {y} = {x*y}")
# You can also limit the number of digits after the decimal point:
print(f"{x} / {y} = {x/y:.2f}")

In [9]:
# Task 2: 
# Switch the model to the basic neural network and train again.

In [ ]:
# Task 3
# Modify the function below to actually use the sex, age and pclass, not just zeros.
# Would you have survived?
def predict(sex: int, age: int, pclass: int):
    model.eval()
    features = torch.tensor([0, 0, 0], dtype=torch.float32)
    print(features)
    pred = model(features)
    print(pred)

predict(sex=0, age=35, pclass=3)

# Optional Task 3.1 (torch, python):
# make the output pretty and print whether the individual likely survived or died.

In [11]:
# Task 4:
# Play around with the hyperparamters learning_rate, batch_size and epochs; also compare the linear regressor and neural network.
# Tip: They are at the top of the cells
# Try to improve the accuracy of the model!
# Tip: Spend less than 15 minutes on this task.

In [12]:
# Task 5:
# Add the "Fare" as a feature. (Fare = ticket price)
# Tip: you have to increase the in_features of your model to allow for the additional input.
# Is there a way to automatically adjust the in_features?

In [13]:
# Task 6:
# Increase the hidden size of the neural network.

# Task 6.1:
# If you haven't done so already: make the hidden size a parameter for the __init__, analog to in_features.

In [14]:
# Task 7:
# Use AdamW instead of SGD as an optimizer

In [15]:
# Optional Task: (torch, python)
# Write a class for a deep neural network
# Tip: A deep neural network contains at least 2 Hidden layers, one more than the BasicNeuralNetwork
# Tip: Just copy the code from the Basic Neural network and add one layer

In [ ]:
# Optional Task: (torch, ml)
# Look into what dropout is, example below
# Add dropout to the model. Experiment with different probabilities.
# Tip: Do not apply dropout on the inputs or the outputs, but after a linear layer.
prob = 0.5
dropout = torch.nn.Dropout(p=prob)
tensor = torch.tensor([[0.0, 1.0, 2.0, 3.0, 4.0, 5.0]])
for i in range(10):
    print(dropout(tensor))

In [17]:
# Optional Task: (python, torch)
# Write a Neural Network class that takes a list of integers as an argumnet and constructs a corresponding neural network. 
# The first number is the number of inputs, the last the number of outputs and the other numbers the size of the hidden layers in between.

In [18]:
# Optional Task (torch)
# Calculate accuracy with torch directly, without multiclass_accuracy
# Tip: use argmax
# Ensure that your implementation scales to multiclass problems

In [19]:
# Optional Task: (polars, python, torch)
# Tip: This is one of the harder tasks.
# Try to include other features in your model, including string features.
# Try to use one-hot-encoding for string features.
# Tip: Modify the titanic notebook.

In [ ]:
# Optional Task (torch)
# Train a LinearRegressor and try to interpret the weights.
# Can you do the same for the neural network or even the deep neural network?

for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)

In [21]:
# Optional Task (python, polars):
# Find another Dataset with similarly structured/tabular data and train a model on the new dataset.

In [22]:
# Optional Task (python and/or polars)
# Do some data analysis on the training data and figure out what predicts whether a passanger survived.
# Try to use polars as much as possible, and don't dare to use pandas.
# Do your results corespond with the model paramteres of the LinearRegressor?

# Optional Task (python)
# Try to visualize your results (for example with pyplot)

In [23]:
# Optional Task (ml, python)
# Implement early stopping

In [24]:
# Optional Task (ml, torch)
# Look into the parameters of AdamW and play around with them

In [25]:
# Optional Task (torch, ml)
# Use a learning rate shedule

In [26]:
# Optional Task (torch)
# clalculate other metrics than simple accuracy. E. g. precicsion, recall, balanced accuracy etc.
# Use pytroch directly

In [27]:
# Optional Task (torch)
# Rewrite the notebook in such a way that the model only has one output, using sigmoid instead of softmax.
# Tip: create a copy of this notebook